# Indirect impact of river flow on electricity price

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression

from scripts.utils import scale_font_latex

## Load harmonized data

In [3]:
TRUE_START = pd.Timestamp("20180101T00", tz="utc")
TRUE_END = pd.Timestamp("20231231T23", tz="utc")
true_years = f"{TRUE_START.year}-{TRUE_END.year}"
COUNTRY_CODE = "FR"


data = pd.read_csv(
    f"../data/processed/combined_data/data_selected_{COUNTRY_CODE}_{true_years}.csv",
    parse_dates=["timestamp"],
).set_index("timestamp")
# Drop NaNs
data = data.dropna()
data.index = pd.to_datetime(data.index, utc="utc")
data["river_flow_mean"] = data["river_flow_mean"] / 1000  # convert to m^3/s

## indirect impact of river flow on price

In [ ]:
scale_font_latex(1.25)
start = TRUE_START
end = TRUE_END


data_selected = data.loc[start:end]
# Calculate correlation
correlation = np.corrcoef(data_selected["river_flow_mean"], data_selected["price_da"])[
    0, 1
]
print("Correlation with price_da:", correlation)


# linear fit:
x_val = data_selected["river_flow_mean"].values.reshape((-1, 1))
y_val = data_selected["price_da"].values.reshape((-1, 1))
linear_model = LinearRegression().fit(x_val, y_val)
slope = linear_model.coef_[0][0]
intercept = linear_model.intercept_[0]
print("Slope:", slope)
print("Intercept:", intercept)


fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(
    x_val,
    linear_model.predict(x_val),
    color="black",
    label=f"slope: {slope:.3f}  EUR/MWh$\cdot$s/$m^3$",
    alpha=0.7,
    linewidth=2,
)
# Set labels
ax.set_xlabel("River flow ($m^3$/s)")
ax.set_ylabel("Price FR (EUR/MWh)")

plt.ylim(0, 1000)
plt.scatter(
    data_selected["river_flow_mean"],
    data_selected["price_da"],
    s=1,
    c=sns.color_palette("colorblind")[0],
)
plt.legend()
fig_dir = "../reports/figures/indirect_effects"
os.makedirs(fig_dir, exist_ok=True)
fig.savefig(fig_dir + f"/price_vs_river_flow.png", dpi=450, bbox_inches="tight")
plt.show()


# Calculate correlation
correlation = np.corrcoef(
    data_selected["river_flow_mean"], data_selected["agg_net_export"]
)[0, 1]
print("Correlation with agg_net_export:", correlation)

# Calculate correlation
correlation = np.corrcoef(data_selected["river_flow_mean"], data_selected["gas_price"])[
    0, 1
]
print("Correlation with gas price:", correlation)

# Calculate correlation
correlation = np.corrcoef(
    data_selected["river_flow_mean"], data_selected["carbon_price"]
)[0, 1]
print("Correlation with carbon price:", correlation)

# Calculate correlation
correlation = np.corrcoef(data_selected["price_da"], data_selected["gas_price"])[0, 1]
print("Correlation price with gas price:", correlation)

In [ ]:
scale_font_latex(1.25)
start = TRUE_START
end = TRUE_END


data_selected = data.loc[start:end]
# Calculate correlation
correlation = np.corrcoef(
    data_selected["river_flow_mean"], data_selected["agg_net_export"]
)[0, 1]
print("Correlation with price_da:", correlation)


# linear fit:
x_val = data_selected["river_flow_mean"].values.reshape((-1, 1))
y_val = data_selected["agg_net_export"].values.reshape((-1, 1))
linear_model = LinearRegression().fit(x_val, y_val)
slope = linear_model.coef_[0][0]
intercept = linear_model.intercept_[0]
print("Slope:", slope)
print("Intercept:", intercept)


fig, ax = plt.subplots(1, 1, figsize=(16, 9))
ax.plot(
    x_val,
    linear_model.predict(x_val),
    color="black",
    label=f"slope: {slope:.3f}  MW$\cdot$s/$m^3$",
    alpha=0.7,
    linewidth=2,
)
# Set labels
ax.set_xlabel("River flow ($m^3$/s)")
ax.set_ylabel("Net exports (MW)")

# plt.ylim(-15, 20)
plt.scatter(
    data_selected["river_flow_mean"],
    data_selected["agg_net_export"],
    s=1,
    c=sns.color_palette("colorblind")[1],
)
plt.legend()
fig_dir = "../reports/figures/indirect_effects"
os.makedirs(fig_dir, exist_ok=True)
fig.savefig(fig_dir + f"/net_exports_vs_river_flow.png", dpi=450, bbox_inches="tight")
plt.show()